**Check for cuda**

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

**Down Data**

In [3]:
# import requests
# import zipfile
# from pathlib import Path

# data_path = Path("data/")
# image_path = data_path / "pizza_steak_sushi"

# if image_path.is_dir():
#     print(f"{image_path} directory exists.")
# else:
#     print(f"Did not find {image_path} directory, creating one...")
#     image_path.mkdir(parents=True, exist_ok=True)

#     with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
#         request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
#         print("Downloading pizza, steak, sushi data...")
#         f.write(request.content)

#     with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
#         print("Unzipping pizza, steak, sushi data...")
#         zip_ref.extractall(image_path)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
unzip = 0
if unzip == 0:
  !unzip /content/drive/MyDrive/Animal_dataset/VGG_16.zip -d /content/animal_dataset
  unzip += 1
else:
  print("already zip")

**Build Custome Dataset**

In [ ]:
# train_dir = image_path / "train"
# test_dir = image_path / "test"

# train_dir, test_dir

In [ ]:
train_dir = '/content/animal_dataset/VGG_16/train'
test_dir = '/content/animal_dataset/VGG_16/test'

train_dir, test_dir

In [ ]:
import os
import pathlib
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, datasets
from torchvision.transforms import Resize, ToTensor, Compose, Normalize, RandomHorizontalFlip
from torchvision.models import resnet34, ResNet34_Weights
from PIL import Image
from typing import Tuple, Dict, List
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import argparse
import cv2
import shutil



In [ ]:
# Make function to find classes in target directory
def find_classes(directory: str) -> Tuple[List[str], Dict[str, int]]:
    """Finds the class folder names in a target directory.

    Assumes target directory is in standard image classification format.

    Args:
        directory (str): target directory to load classnames from.

    Returns:
        Tuple[List[str], Dict[str, int]]: (list_of_class_names, dict(class_name: idx...))

    Example:
        find_classes("food_images/train")
        >>> (["class_1", "class_2"], {"class_1": 0, ...})
    """
    classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())

    if not classes:
        raise FileNotFoundError(f"Couldn't find any classes in {directory}.")

    class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
    return classes, class_to_idx

In [ ]:
from torch.utils.data import Dataset

class ImageFolderCustom(Dataset):
  def __init__(self, root_dir, is_train, transform):
    if is_train:
      data_path = os.path.join(root_dir, "train")
    else:
      data_path = os.path.join(root_dir, "test")
    self.root_dir = root_dir
    self.categories = ['cow', 'horse', 'sheep']
    self.image_paths = []
    self.labels = []
    self.transform = transform
    for index, category in enumerate(self.categories):
      subdir_path = os.path.join(data_path, category)
      for file_name in os.listdir(subdir_path):
          self.image_paths.append(os.path.join(subdir_path,subdir_path, file_name))
          self.labels.append(index)

  def __len__(self) -> int:
      "Returns the total number of samples."
      return len(self.image_paths)

  def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
      "Returns one sample of data, data and label (X, y)."
      image = Image.open(self.image_paths[index]).convert("RGB")
      class_idx = self.labels[index]
      if self.transform:
          image = self.transform(image)
      return image, class_idx

In [ ]:
root_dir = r'/content/animal_dataset/VGG_16'

In [ ]:
def data_loader():
  train_transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    RandomHorizontalFlip(p=0.5),
    Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010])
    ])
  test_transform = Compose([
    Resize((224, 224)),
    ToTensor()
    ])

  train_dataset = ImageFolderCustom(root_dir=root_dir, is_train=True, transform=train_transform)
  train_dataloader = DataLoader(
      dataset = train_dataset,
      batch_size = 32,
      num_workers = 4,
      shuffle = True,
      drop_last = False
  )
  test_dataset = ImageFolderCustom(root_dir=root_dir, is_train=False, transform=test_transform)
  test_dataloader = DataLoader(
        dataset = test_dataset,
        batch_size = 32,
        num_workers = 2,
        shuffle = False,
        drop_last = False
  )
  return train_dataloader, test_dataloader

train_dataloader, test_dataloader = data_loader()

In [ ]:
# train_transform = Compose([
#     Resize((224, 224)),
#     ToTensor(),
#     RandomHorizontalFlip(p=0.5),
#     Normalize(
#         mean=[0.4914, 0.4822, 0.4465],
#         std=[0.2023, 0.1994, 0.2010])
#     ])
# train_dataset = ImageFolderCustom(root_dir=root_dir, is_train=True, transform=train_transform)
# images_neduma,labels_neduma = train_dataset.__getitem__(200)
# print(images_neduma.shape)
# print(labels_neduma)

In [ ]:
# from PIL import Image
# import matplotlib.pyplot as plt
# categories = ['pizza', 'steak', 'sushi']
# data = ImageFolderCustom(root_dir=root_dir,  is_train=True)

# image,label = data.__getitem__(200)

# img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# im_pil = Image.fromarray(img)

# im_np = np.asarray(im_pil)

# plt.imshow(im_np)
# plt.title(categories[label])
# plt.axis('off')
# plt.show()


**MODEL**

In [ ]:
IMAGE_SIZE = 224
NUM_CLASSES = 2

NUM_EPOCHS = 100
NUM_WORKERS = 4
BATCH_SIZE = 32
LEARNING_RATE = 0.001

CONV_KERNEL = 3
CONV_STRIDE = 1
CONV_PADDING = 1
MP_KERNEL = 2
MP_STRIDE = 2
MP_PADDING = 0
VGG16_archite = [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]

In [ ]:
class VGG(nn.Module):
  def __init__(self, in_channels=3, num_classes=None):
    super(VGG, self).__init__()
    self.in_channels = in_channels
    self.features = self.create_conv_layers(VGG16_archite)
    # self.image_size = max_pooling_output_size_for_fc(H=IMAGE_SIZE, n=len())
    self.avgpool = nn.AdaptiveAvgPool2d((7,7))
    self.classifier = nn.Sequential(
        nn.Linear(512*7*7, 4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 1000),
        nn.ReLU(),
        nn.Linear(1000,512),
        nn.ReLU(),
        nn.Linear(512, num_classes),
    )

  def forward(self,x):
    x = self.features(x)
    x = x.reshape(x.shape[0], -1)
    x = self.classifier(x)
    return x

  def create_conv_layers(self, archite):
    layers = []
    in_channels = self.in_channels
    for x in archite:
      if type(x) == int:
        out_channels = x
        layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(CONV_KERNEL,CONV_KERNEL), stride=(CONV_STRIDE,CONV_STRIDE), padding=(CONV_PADDING,CONV_PADDING)),
                  #  nn.BatchNorm2d(x),
                   nn.ReLU()]
        in_channels = x
      elif x == "M":
        layers += [nn.MaxPool2d(kernel_size=(MP_KERNEL,MP_KERNEL), stride=(MP_STRIDE,MP_STRIDE))]
    return nn.Sequential(*layers)


In [ ]:
import torch
from torchvision.models import vgg16, VGG16_Weights

# Ensure that device is defined (e.g., 'cpu' or 'cuda')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the VGG16 model with pretrained weights
model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1).to(device)
print("Pytorch VGG16 - achritecture")
print(model)
print('----------------------------------')

print("My VGG16 - achritecture")
model = VGG(in_channels=3, num_classes=100).to(device=device)
state_dict = torch.load(r'/root/.cache/torch/hub/checkpoints/vgg16-397923af.pth')
model.load_state_dict(state_dict, strict= False)
print(model)

**LOST - OPTIMIZER**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

**Train/Test Phrase**

In [ ]:
stored_dir = r'/content/trainningPhrase/'
check_point = r'/content/check_point'

In [ ]:
def trainning(stored_dir,check_point):

  writer = SummaryWriter('content/runs')
  best_acc = 0
  model.to(device)
  if os.path.isdir(stored_dir):
        shutil.rmtree(stored_dir)
  os.makedirs(stored_dir)
  if not os.path.isdir(check_point):
        os.makedirs(check_point)
  for epoch in range(NUM_EPOCHS):
    model.train()
    progress_bar = tqdm(train_dataloader, colour="yellow")
    for i, (images, labels) in enumerate(progress_bar):
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            loss = criterion(output, labels)
            progress_bar.set_description("Epoch {}/{}. Loss {:0.4f}".format(epoch + 1, NUM_EPOCHS, loss))
            writer.add_scalar("Train/loss", loss, epoch * len(train_dataloader) + i)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    model.eval()
    all_losses = []
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for i, (images, labels) in enumerate(test_dataloader):
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            predictions = torch.argmax(output, dim=1)
            loss = criterion(output, labels)
            all_losses.append(loss.item())
            all_labels.extend(labels.tolist())
            all_predictions.extend(predictions.tolist())
        loss = np.mean(all_losses)
        accuracy = accuracy_score(all_labels, all_predictions)
        conf_matrix = confusion_matrix(all_labels, all_predictions)
        print("Epoch {}/{}. Loss {:0.4f}. Acc {:0.4f}".format(epoch + 1, NUM_EPOCHS, loss, accuracy))
        writer.add_scalar("Test/loss", loss, epoch)
        writer.add_scalar("Test/Accuracy", accuracy, epoch)
        checkpoint = {
            "model_state_dict": model.state_dict(),
            "epoch": epoch,
            "optimizer_state_dict": optimizer.state_dict()
        }
        torch.save(checkpoint, os.path.join(check_point, "last.pt"))
        if accuracy > best_acc:
            torch.save(checkpoint, os.path.join(check_point, "best.pt"))
            best_acc = accuracy

In [ ]:
def main():
  trainning(stored_dir,check_point)

In [ ]:
main()